In [1]:
import pandas as pd
import pyodbc

In [5]:
username = 'vigrose'
password = 'Hufflepuff10946'
s = "DSN=PRDDW; UID={}; PWD={}".format(username, password)
AMAEDW = pyodbc.connect(s)

In [6]:
ppd = pd.read_csv('../../Data/Scorecard/ppd_data_20200704.csv', low_memory=False)

In [7]:
query = \
    """ 
    SELECT DISTINCT
    P.ME,
    L.LIC_NBR
    FROM AMAEDW.LIC L, AMAEDW.PARTY_ID_TO_ME_VW P 
    WHERE
    L.STATE_ID = 244
    AND
    L.PARTY_ID = P.PARTY_ID
    AND
    L.DELETE_IND = 'N'
    AND
    L.THRU_DT IS NULL
    """
nc_lic = pd.read_sql(con=AMAEDW, sql=query)
nc_lic.head()

,ME,LIC_NBR
0,00102000030,200201481
1,00102000129,200201219
2,00102000196,2007-00835
3,00102000200,2005-01659
4,00102000242,2010-01708


In [8]:
len(nc_lic)

86736

In [9]:
def fix_me(me_list):
    nums = []
    for num in me_list:
        num = str(num)
        num = num.replace('.0', '')
        if len(num) == 10:
            num = '0' + num
        elif len(num) == 9:
            num = '00' + num
        elif len(num) == 8:
            num = '000' + num
        nums.append(num)
    return nums
ppd['ME']=fix_me(list(ppd['ME']))

In [10]:
pd.merge(ppd[['ME','FIRST_NAME','LAST_NAME']], nc_lic, on='ME').to_csv('../../Data/License/NC_Lic.csv', index=False)

In [11]:
nc_lic.dropna()

,ME,LIC_NBR
0,00102000030,200201481
1,00102000129,200201219
2,00102000196,2007-00835
3,00102000200,2005-01659
4,00102000242,2010-01708
...,...,...
86731,96501870037,9601638
86732,96501880059,200200552
86733,96501890038,2009-02099
86734,96501960044,2016-02379


In [12]:
NC = pd.merge(ppd[['ME','FIRST_NAME','LAST_NAME']], nc_lic, on='ME')

In [19]:
nc_lic.groupby('LIC_NBR').count().sort_values('ME', ascending =False)

,ME
LIC_NBR,
,12265
36831,3
2005-00468,3
2016-01795,3
2007-01483,3
...,...
2010-01904,1
2010-01903,1
2010-01901,1


In [14]:
NC

,ME,FIRST_NAME,LAST_NAME,LIC_NBR
0,00102000030,MARK,JACKSON,200201481
1,00102000129,CORY,BEAN,200201219
2,00102000196,REBECCA,SUGG,2007-00835
3,00102000200,CHARLES,BLAKE,2005-01659
4,00102000242,JOSEPH,MOBLEY,2010-01708
...,...,...,...,...
75661,96501870037,SYED,HUSSAIN,9601638
75662,96501880059,NASEEM,ISMAIL,200200552
75663,96501890038,BETSY,PANICKER,2009-02099
75664,96501960044,MWIZA,GAUSI,2016-02379


In [1]:
import pandas as pd

In [2]:
df_1 = pd.read_csv('../../Data/License/NC_License_Status_2021-11-30_01.csv')
df_2 = pd.read_csv('../../Data/License/NC_License_Status_2021-11-30_2.csv')
df_3 = pd.read_csv('../../Data/License/NC_License_Status_2021-11-30_3.csv')
# df_4 = pd.read_csv('../../Data/License/NC_License_Status_2021-06-04_1.csv')
# df_5 = pd.read_csv('../../Data/License/NC_License_Status_2021-04-16_2.csv')
# df_6 = pd.read_csv('../../Data/License/NC_License_Status_2021-04-17.csv')

In [3]:
file = pd.concat([df_1, df_2, df_3])

In [3]:
file = pd.concat([df_1, df_2])

In [4]:
file = pd.read_csv('../../Data/License/NC_License_Status_2021-02-11.csv')

In [5]:
og_file = pd.read_csv('../../Data/License/NC_FULL_FILE_NUM_ONLY_2021-11-29.csv')

In [12]:
july_file = pd.merge(og_file, file, on= 'License_Number')

In [9]:
pd.merge(og_file, file, on= 'License_Number').to_csv('../../Data/License/NC_License_Status_2021-07-14.csv', index=False)

In [10]:
og_file[~og_file.License_Number.isin(file.License_Number)]

,License_Number


In [6]:
len(og_file)

44129

In [7]:
len(file)

44129

In [8]:
file[file.License_Type=='Not found']

,License_Number,License_Type
6563,2017-02476,Not found


In [10]:
june_file = pd.read_csv('../../Data/License/NC_License_Status_2021-06-04.csv')

In [13]:
pd.merge(june_file, july_file, on='License_Number')

,License_Number,License_Type_x,License_Type_y
0,2007-01286,MD,Not found
1,2018-02560,MD,MD
2,2013-02286,DO,DO
3,38886,MD,MD
4,38647,MD,MD
...,...,...,...
43529,2015-01273,MD,MD
43530,2021-00527,MD Limited Emergency License,MD Limited Emergency License
43531,2015-01831,MD,MD
43532,2021-01133,MD,MD


In [16]:
july_file[july_file.License_Number.isin(june_file.License_Number)==False]

,License_Number,License_Type
2,2021-02107,MD
71,2011-01423,MD
385,2021-01992,MD
413,2021-01978,MD
452,2021-02080,DO Limited Emergency License
...,...,...
43741,2021-02085,DO
43781,2021-02287,MD
43824,2021-02005,MD
43890,2021-02030,MD


In [17]:
may_file = pd.read_csv('../../Data/License/NC_License_Status_2021-05-12.csv')

In [18]:
june_file[june_file.License_Number.isin(may_file.License_Number)==False]

,License_Number,License_Type
192,2021-01770,MD
254,2021-01823,MD
345,2021-01685,MD
389,2021-01609,MD
414,2021-01834,MD
...,...,...
42041,2021-01799,MD
42463,2021-01659,MD
42716,2021-01748,MD
43088,2021-01790,DO


In [3]:
scraped = pd.read_csv('../../Data/License/NC_License_Status_2022-06-14_012125613793.csv')
og = pd.read_csv('../../Data/License/NC_2022-06-14.csv')

In [4]:
old_1 = pd.read_csv('../../Data/License/NC_License_Status_2022-05-04.csv')
old_2 = pd.read_csv('../../Data/License/NC_License_Status_2022-05-05.csv')
old_3 = pd.read_csv('../../Data/License/NC_License_Status_2022-05-06.csv')
test_old = pd.concat([old_1, old_2, old_3, scraped])

In [21]:
test_old = pd.concat([old_1, old_2, old_3])

In [22]:
test_old.to_csv('../../Data/License/NC_License_Status_2022-06-14_2.csv', index=False)

In [10]:
public_flags = ['DO Faculty Limited/WFU & Affiliated Institutions',
            'DO Limited Emergency License',
            'DO Retired Limited Volunteer',
            'MD Faculty Limited',
            'MD Faculty Limited/Duke & Affiliated Institutions',
            'MD Faculty Limited/ECU & Affiliated Institutions',
            'MD Faculty Limited/UNC & Affiliated Institutions',
            'MD Faculty Limited/WFU & Affiliated Institutions',
            'MD Limited Emergency License',
            'MD Limited License Volunteer',
            'MD Retired Limited Volunteer',
            'MD Special Permit License',
            'MD Special Volunteer',
            'MD Volunteer Limited']
for lic_type in test_old.License_Type.unique():
    split_type = lic_type.split(' ')
    if len(split_type)>1 and split_type[0] in ['MD','DO']:
        if lic_type not in public_flags:
            print(f'New License Type parameter added: {lic_type}')
            public_flags.append(lic_type)

New License Type parameter added: MD Faculty Limited/Duke/UNC & Affiliated Institutions


In [13]:
filey = pd.read_csv('../../Data/License/MD Active Type Fixed - Updated.txt', sep = '\t')

In [18]:
filey.groupby('License_Type').count()

,Last_Name,First_Name,MiddleName,Suffix,License_Number,LicenseIssueDate,LicenseExpDate,Status,AnnualRenewalDate,PrimaryAreaofPractice,...,City,State,County,Zip_Code_Public,MedSchool,MedSchoolState,Medical_School_Graduation_Year,Limitation,Race,Gender
License_Type,,,,,,,,,,,,,,,,,,,,,
DO,3468,3468,3178,136,3468,3468,0,3468,3461,3465,...,3400,3403,2640,2943,3468,3094,3432,0,0,0
MD,39335,39335,34827,2358,39335,39335,0,39335,39292,39137,...,38719,38674,30846,32794,39274,21855,37781,0,0,0


In [32]:
testyy = pd.read_csv('../../Data/License/NC_License_Status_2022-07-11.csv')

In [35]:
testyy[testyy.PublicFlag=='Y']

,Last_Name,First_Name,Middle_Name,Suffix,License_Type,License_Number,LicenseIssueDate,LicenseExpDate,Status,AnnualRenewalDate,...,MedSchoolState,Medical_School_Graduation_Year,SuperLastName,SuperFirstName,SuperMiddleName,SuperSuffix,Limitation,Race,Gender,Primary_Source_Verified
58,Abdulaziz M Alenezi,Fawaz,NaN,NaN,MD,2020-03630,08/04/2020,NaN,Active,12/30/2022,...,NaN,2005.0,NaN,NaN,NaN,NaN,NaN,White,Male,No
106,Abraham,Elizabeth,Moolamannil,NaN,MD,200500145,10/27/2016,NaN,Active,11/09/2022,...,NaN,1973.0,NaN,NaN,NaN,NaN,NaN,Asian: Indian,Female,Yes
176,Adair,Norman,Eugene,NaN,MD,25105,12/01/2021,NaN,Active,01/24/2023,...,NaN,1974.0,NaN,NaN,NaN,NaN,NaN,White,Male,Yes
180,Adam,Soheir,NaN,NaN,MD,2008-01390,07/31/2008,NaN,Active,08/31/2022,...,NaN,1983.0,NaN,NaN,NaN,NaN,NaN,Other racial group not listed,Female,No
207,Adams,John,Stewart,NaN,MD,28724,10/22/2019,NaN,Active,11/07/2022,...,NaN,1988.0,NaN,NaN,NaN,NaN,NaN,White,Male,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42722,Zaas,David,William,NaN,MD,200101304,09/02/2020,NaN,Active,10/14/2022,...,IL,1998.0,NaN,NaN,NaN,NaN,NaN,White,Male,Yes
42879,Zendel,Alexey,NaN,NaN,MD,2020-03584,08/02/2021,NaN,Active,07/31/2022,...,NaN,2004.0,NaN,NaN,NaN,NaN,NaN,White,Male,No
42886,Zeok,Suzanne,Springer,NaN,MD,25762,08/07/2018,NaN,Active,08/02/2023,...,NaN,1970.0,NaN,NaN,NaN,NaN,NaN,White,Female,Yes
42934,Zhao,Jin,NaN,NaN,MD,2006-01753,10/15/2021,NaN,Active,10/28/2022,...,NaN,1982.0,NaN,NaN,NaN,NaN,NaN,Asian: Chinese,Female,Yes


In [2]:
delta = pd.read_csv('../../Data/License/NC_License_Status_2022-08-08_12.csv')
july = pd.read_csv('../../Data/License/NC_License_Status_2022-07-11_1.csv')
og_file = pd.read_csv('../../Data/License/NC_2022-08-08.csv')

In [20]:
all_that = pd.concat([delta, july[['License_Number','License_Type']]])

In [16]:
july.License_Type = lic_list

In [14]:
['MD Faculty Limited' for x in july.PublicFlag if x == 'Y' else '']

['MD Faculty Limited',
 'MD Faculty Limited',
 'MD Faculty Limited',
 'MD Faculty Limited',
 'MD Faculty Limited',
 'MD Faculty Limited',
 'MD Faculty Limited',
 'MD Faculty Limited',
 'MD Faculty Limited',
 'MD Faculty Limited',
 'MD Faculty Limited',
 'MD Faculty Limited',
 'MD Faculty Limited',
 'MD Faculty Limited',
 'MD Faculty Limited',
 'MD Faculty Limited',
 'MD Faculty Limited',
 'MD Faculty Limited',
 'MD Faculty Limited',
 'MD Faculty Limited',
 'MD Faculty Limited',
 'MD Faculty Limited',
 'MD Faculty Limited',
 'MD Faculty Limited',
 'MD Faculty Limited',
 'MD Faculty Limited',
 'MD Faculty Limited',
 'MD Faculty Limited',
 'MD Faculty Limited',
 'MD Faculty Limited',
 'MD Faculty Limited',
 'MD Faculty Limited',
 'MD Faculty Limited',
 'MD Faculty Limited',
 'MD Faculty Limited',
 'MD Faculty Limited',
 'MD Faculty Limited',
 'MD Faculty Limited',
 'MD Faculty Limited',
 'MD Faculty Limited',
 'MD Faculty Limited',
 'MD Faculty Limited',
 'MD Faculty Limited',
 'MD Facult

In [15]:
lic_list = []
for row in july.itertuples():
    if row.PublicFlag == 'Y' and row.License_Type == 'MD':
        lt = 'MD Faculty Limited'
    elif row.PublicFlag == 'Y' and row.License_Type == 'DO':
        lt = 'DO Limited Emergency License'
    else:
        lt = row.License_Type
    lic_list.append(lt)

In [23]:
juul = july[['License_Number','License_Type']]

In [28]:
juul[juul.License_Number.isin(all_that.License_Number)].to_csv('../../Data/License/NC_License_Status_2022-08-08_3.csv', index=False)

In [25]:
len(all_that)

43390

In [ ]:
def find_all_scraped_data():
    out_dir = os.environ.get('LICENSE_DIR')
    this_month = date.today().strftime("%Y-%m")
    # this_month = '2021-11'
    file_list = []
    for file in os.listdir(out_dir):
        if file.startswith(f"NC_License_Status_{this_month}"):
            file_list.append(f'{out_dir}{file}')
    return file_list